In [ ]:
import os
import time
import json
from neo4j import GraphDatabase
import json
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 1. 调用大模型 api

星火、智谱 8192 toekn，通义长文本 28k tokens

## 1.1 智谱清言 glm-4

配置密钥信息，将前面获取到的 API key 设置到 .env 文件中的 ZHIPUAI_API_KEY 参数，然后运行以下代码加载配置信息。

模型的最大输入长度 512 字符数，输出 8192 token

[智谱 api 文档](https://maas.aminer.cn/dev/api#glm-4)

智谱的调用传参和其他类似，也需要传入一个 messages 列表，列表中包括 role 和 prompt。我们封装如下的 `get_completion` 函数，供后续使用。

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key=os.environ["ZHIPUAI_API_KEY"])

def gen_glm_params(prompt):
    '''
    构造 GLM 模型请求参数 messages
    请求参数：prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages

def get_completion(prompt, model="glm-4", temperature=0.01, max_tokens=8192):
    '''
    获取 GLM 模型调用结果
    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 glm-4，也可以按需选择 glm-3-turbo 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    messages = gen_glm_params(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

## 1.2 调用文心一言

In [ ]:
import qianfan

def gen_wenxin_messages(prompt):
    '''
    构造文心模型请求参数 messages
    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="ERNIE-Bot", temperature=0.01, max_output_tokens=4096):
    '''
    获取文心模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 ERNIE-Bot，也可以按需选择 ERNIE-Bot-4 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    chat_comp = qianfan.ChatCompletion()
    message = gen_wenxin_messages(prompt)

    resp = chat_comp.do(messages=message, 
                        model=model,
                        temperature = temperature,
                        system="你是一个对电子商务领域有深入了解的专家")  # 人设

    return resp["result"]

## 1.3 讯飞星火

In [ ]:
import os
import sys
import sparkAPI


def gen_spark_params(model):
    '''
    构造星火模型请求参数
    '''

    spark_url_tpl = "wss://spark-api.xf-yun.com/{}/chat"
    model_params_dict = {
        # v1.5 版本
        "v1.5": {
            "domain": "general", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v1.1") # 云端环境的服务地址
        },
        # v2.0 版本
        "v2.0": {
            "domain": "generalv2", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v2.1") # 云端环境的服务地址
        },
        # v3.0 版本
        "v3.0": {
            "domain": "generalv3", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.1") # 云端环境的服务地址
        },
        # v3.5 版本
        "v3.5": {
            "domain": "generalv3.5", # 用于配置大模型版本
            "spark_url": spark_url_tpl.format("v3.5") # 云端环境的服务地址
        }
    }
    return model_params_dict[model]


def get_completion(prompt, model="v3.5", temperature = 0.01, max_tokens=8192):
    '''
    获取星火模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 v3.5，也可以按需选择 v3.0 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    response = sparkAPI.main(
        appid=os.environ["SPARK_APPID"],
        api_secret=os.environ["SPARK_API_SECRET"],
        api_key=os.environ["SPARK_API_KEY"],
        gpt_url=gen_spark_params(model)["spark_url"],
        domain=gen_spark_params(model)["domain"],
        query=prompt,
        max_tokens=max_tokens
    )
    return response

## 1.4 通义千问

前提条件
已开通服务并获得API-KEY：开通DashScope并创建API-KEY。
https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key

[灵积控制台 模型 计费管理](https://dashscope.console.aliyun.com/billing)

[通义 api 文档](https://help.aliyun.com/zh/dashscope/developer-reference/api-details)

In [ ]:
import requests
import os
from dotenv import dotenv_values

api_key = dotenv_values('.env')['DASHSCOPE_API_KEY']
# api_key = os.getenv("DASHSCOPE_API_KEY")
url = 'https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation'
headers = {'Content-Type': 'application/json',
           'Authorization':f'Bearer {api_key}'}

def get_completion(prompt, model="qwen-max-longcontext", max_tokens=2000):
    '''
    max_tokens  其中qwen-turbo最大值和默认值为1500，qwen-max、qwen-max-1201 、qwen-max-longcontext 和 qwen-plus最大值和默认值均为2000。
    model       目前可选择qwen-turbo、qwen-plus、qwen-max、qwen-max-0403、qwen-max-0107、qwen-max-1201和qwen-max-longcontext
    '''
    body = {
        'model': model,
        "input": {
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        },
        "parameters": {
            "result_format": "message",
            "max_tokens":max_tokens,
            "enable_search":True

        }
    }

    response = requests.post(url, headers=headers, json=body)
    response_ = response.json()
    return response_['output']['choices'][0]['message']['content']
    # ['output']['choices'][0]['message']['content']
    # .choices[0].message.content
    # ['output']['choices'][0]['message']
    # output.choices[x].message.content

In [ ]:
get_completion("你是谁？")

# 2. 构建知识图谱

## 2.1 获取数据

In [ ]:
import re
from docx import Document
import os

# 定义文件夹路径，使用相对路径
folder_path = r'./database/课程/part_textbook'

# 初始化一个空字符串用于存储所有文档的内容
str_content = ""

# 确保路径存在
if not os.path.exists(folder_path):
    print("指定的文件夹路径不存在，请检查路径是否正确。")
else:
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):  # 确保是Word文档
            file_path = os.path.join(folder_path, filename)
            doc = Document(file_path)  # 读取Word文档
            for para in doc.paragraphs:  # 遍历文档中的所有段落
                # 将段落文本添加到字符串中，移除前后空格
                str_content += para.text.strip()

    # 使用正则表达式移除所有不可见字符和换行符
    str_content = re.sub(r'[^\w\s]', '', str_content)
    str_content = re.sub(r'\s+', ' ', str_content)  # 将多个空格替换为一个空格

    # 打印结果，此时不应该包含换行符
    print(str_content)

## 2.2 调用大模型

**一本教材大约需要 500 万 token**

输出：
"""
    ``` json
    [{
        "node_id": "EC_K_0001",
        "name_zh": "商业模式",
        "name_en": "Business Model",
        "explanation": "企业为客户和利益相关方创造价值所采用的商业逻辑或系统。"
    },    {
        "node_id": "EC_K_0002",
        "name_zh": "价值主张",
        "name_en": "value proposition",
        "explanation": "企业为客户提供的独特价值或利益。"
    },    {
        "node_id": "EC_K_0010",
        "name_zh": "O2O",
        "name_en": "",
        "explanation": "Online to Offline，线上到线下，指线上营销带动线下消费。"
    },    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0002",
        "relation": "include"
    },    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0010",
        "relation": "include"
    }]
```
```json
[
    {
        "node_id": "EC_K_0001",
        "name_zh": "商业模式",
        "name_en": "business model",
        "explanation": "企业通过创造价值而获取收益所采取的一系列活动，表明了公司在价值链中所处的位置"
    },
    {
        "node_id": "EC_K_0002",
        "name_zh": "价值主张",
        "name_en": "value proposition",
        "explanation": "面向客户的价值，商业模式需要回答客户价值是什么"
    },

"""

In [ ]:
# 定义一个空列表来存储关键词
list_str_keywords = []

# 打开文件并逐行读取
with open(r'./database/课程/part_keywords/all_keywords.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # 使用split()方法根据制表符分割每一行，然后取第一个元素作为关键词
        keyword = line.strip().split('\t')[0]
        # 将关键词添加到列表中
        list_str_keywords.append(keyword)

# 打印结果以验证
print(list_str_keywords)

In [ ]:
# 初始化一个空列表来存储返回值
completion_results = []

# 定义每次截取的字符长度
chunk_size = 400

# 设置请求间隔时间（秒）
interval = 20

# 初始化循环次数计数器
counter = 649


# 循环字符串，每次截取 500 个字符
for chunk_start in range((counter*chunk_size), len(str_content), chunk_size):
    # 截取字符串的子集
    chunk = str_content[chunk_start: chunk_start + chunk_size]
    # 调用函数处理每个字符串块
    prompt_text_entity = """你是一个对电子商务领域有深入了解的专家。请基于电子商务领域的百科词条以及这里提供的关键词 %s，从文本中提取所有关于电子商务的概念作为实体，并获取实体的属性（中文名name_zh、英文名name_en、定义explanation）""" % (list_str_keywords)
    result_entity = get_completion(prompt_text_entity + "待处理文本为：" + chunk)
    
    prompt_text_relation = """根据 %s 和输入的文本 %s ，识别实体之间的所有关系（include表示知识点A_id包含知识点B_id；order表示知识点A_id学完才有基础学习知识点B_id；relate表示知识点A_id与B_id有关，但是非include和order的关系）。输出为json格式，不要含注释。
    请参考我给出的示例：
    <example>
    待处理文本：价值体现或价值主张(value proposition)是确定一个企业的产品或者服务如何满足客户的需求，它是企业商业模式的核心。商业模式就是企业通过创造价值而获取收益所采取的一系列活动，它表明了公司在价值链中所处的位置。任何商业模式都要清楚他们的赢利模式和价值主张。
    输出：
    [    {
            "node_id": "EC_K_0001",
            "name_zh": "价值体现",
            "name_en": "value proposition",
            "explanation": "确定一个企业的产品或者服务如何满足客户的需求"
        },    {
            "node_id": "EC_K_0002",
            "name_zh": "商业模式",
            "name_en": "Business Model",
            "explanation": "企业通过创造价值而获取收益所采取的一系列活动，它表明了公司在价值链中所处的位置"
        },    {
            "A_id": "EC_K_0002",
            "B_id": "EC_K_0001",
            "relation": "include"
        }]
    </example>
    """ % (result_entity, chunk)
    result = get_completion(prompt_text_relation + "待处理文本为：" + chunk)    
    
    # 将返回值追加到列表中
    completion_results.append(result)
    
    # 每循环 30 次执行一次 time.sleep(interval)
    counter += 1
    print("第" + str(counter) + "次循环")

    if counter % 30 == 0:
        time.sleep(interval)
        print(chunk[0:50])

    # if counter == 500:
    #     break

# 打印或处理completion_results列表
print(completion_results)

In [ ]:
# with open(r'./output/课程_3_llmGen.txt', 'w', encoding='utf-8') as file:
with open(r'./output/课程_llmGen.txt', 'a', encoding='utf-8') as file:
    # 使用列表推导式和写入操作，将每个字符串写入文件，并追加换行符
    file.writelines(f"{line}\n" for line in completion_results)

## 2.3 清洗数据
### 2.3.1 去除 `[]` 外的字符

In [ ]:
stack = []
indexes = []
file_path = r'./output/课程_llmGen.txt'
content_json = []
with open(file_path, 'r', encoding='utf-8') as file:
    # 读取文件内容到字符串变量
    content = file.read()
for i in range(len(content)):
    char_ = content[i]
    if char_ == '[':
        stack.append(content[i])
        indexes.append(i)
    elif char_ == ']':
        if stack:
            stack.pop()
            if not stack:  # 弹出 [ 后，若为空，说明匹配到了完整的 []
                indexes.append(i)
                content_json.append(content[indexes[0]:indexes[-1]] + "]")
                indexes = []
            else:  # 若栈不空，继续寻找匹配的 ]
                indexes.pop()
        else:  # 遇到 ] 但栈为空，表明没有匹配的 [
            print("no match [")
    elif char_ == '`':
        if stack:  # 如果当前有未闭合的 [
            context = content[max(0, i-20):i]  # 获取当前位置前20个字符，如果i小于20，则从开头开始
            print(f"no match ] near: '{context}'")
            indexes = []
            stack = []

print(content_json)

### 2.3.2 重设 node_id，实体和关系中同步修改

输出形如：
```json

    [{
        "node_id": "EC_K_0169",
        "name_zh": "点击收费",
        "name_en": "Pay per click",
        "explanation": "按有效点击广告并链接到相应位置计算一次收费。"
    },
    {
        "node_id": "EC_K_0170",
        "name_zh": "成交收费",
        "name_en": "Pay per action",
        "explanation": "消费者点击广告进入企业并完成商品购买，按交易额收费。"
    },
    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0002",
        "relation": "include"
    },
    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0003",
        "relation": "include"
    }]
```

In [ ]:
k = 1
nodes_combine = []
relations_combine = []
for i in range(len(content_json)):
    data_json = json.loads(content_json[i])
    # 初始化两个空列表，用于存放不同类型的数据
    json_nodes = []
    json_relations = []
    id_mapping = {}

    # 根据数据内容的差异将数据分别存入两个列表
    for item in data_json:
        if "node_id" in item:
            json_nodes.append(item)
        elif "A_id" in item and "B_id" in item:
            json_relations.append(item)
            
    for j in range(len(json_nodes)):
        new_id = f"EC_K_{(k + j):04}"
        id_mapping[json_nodes[j]['node_id']] = new_id
        json_nodes[j]['node_id'] = new_id
        if j == len(json_nodes) - 1:
            k = k + j + 1
            
    # 使用映射更新 relations 中的 A_id 和 B_id
    for relation in json_relations:
        if relation["A_id"] in id_mapping:
            relation["A_id"] = id_mapping[relation["A_id"]]
        if relation["B_id"] in id_mapping:
            relation["B_id"] = id_mapping[relation["B_id"]]
            
    for i in range(len(json_nodes)):
        nodes_combine.append(json_nodes[i])
    for i in range(len(json_relations)):
        relations_combine.append(json_relations[i])

In [ ]:
data_json_rename = nodes_combine + relations_combine
# data_json_rename
with open(r'./output/课程_proc.json', 'w', encoding='utf-8') as file:
    json.dump(data_json_rename, file, indent=4, ensure_ascii=False)

### 2.3.3 实体中增加type属性，关系中替换renlation键为type

输出形如：
```json
    [{
        "node_id": "EC_K_0169",
        "name_zh": "点击收费",
        "name_en": "Pay per click",
        "explanation": "按有效点击广告并链接到相应位置计算一次收费。",
        "type": "knowledge"
    },
    {
        "node_id": "EC_K_0170",
        "name_zh": "成交收费",
        "name_en": "Pay per action",
        "explanation": "消费者点击广告进入企业并完成商品购买，按交易额收费。",
        "type": "knowledge"
    },
    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0002",
        "type": "include"
    },
    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0003",
        "type": "include"
    }]
```

In [ ]:
import json

# 假设 json 文件名为 example.json
filename = r'./output/课程_proc.json'

# 读取文件中的 JSON 数据
with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
for item in data:
    if "node_id" in item:
        item['type'] = 'knowledge'
    if ("A_id" in item) and ("B_id" in item):
        item['type'] = item.pop('relation')

# # 将更新后的数据写回到文件（可选）
with open(r'./output/课程_proc1.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [ ]:
# 假设 json 文件名为 example.json
filename = r'./output/课程_proc1.json'

# 读取文件中的 JSON 数据
with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

## 2.4 合并实体

根据 explanation 做向量相似度计算

合并 node_name ，让 llm 选择一个 name，其余 name 存到 explanation

In [ ]:
import sys
sys.path.append('..')
from text2vec import Similarity

# class Similarity 中有判断 gpu or cpu 的语句
# SBERT（Sentence-BERT）是基于BERT模型的一个变体，专门用于生成句子级别的向量表示
sim_model = Similarity(similarity_type='cosine', embedding_type='sbert')

list_merge = []
list_merge_node_id = []

dict_merge = {}
set_sim = set()

# 比较每一对文本的相似度
for i in range(9):
    for j in range(i + 1, len(data)):
        if ("node_id" in data[i]) and ("node_id" in data[j]):
            sim_1 = sim_model.get_score(data[i]['name_zh'], data[j]['name_zh'])
            sim_2 = sim_model.get_score(data[i]['explanation'], data[j]['explanation'])
            # # 将打印内容写入文件
            # file.write(f"相似度 - {data[i]['name_zh']} 和 {data[j]['name_zh']}: {sim_1:.2f}\n")
            # file.write(f"相似度 - {data[i]['explanation']} 和 {data[j]['explanation']}: {sim_2:.2f}\n")
            # file.write("=================================\n")
            if (sim_1 >= 0.99) or (sim_2 >= 0.95):
                # set_sim.add(data[i]["node_id"])
                set_sim.add(data[j]["node_id"])
    dict_merge['%s' % (data[i]['node_id'])] = set_sim
    set_sim = set()
    list_merge.append(dict_merge)
    dict_merge = {}

In [ ]:
# 创建一个空列表来存储所有的值
all_values = []

# 遍历列表中的每个字典
for dictionary in list_merge:
    # 遍历字典中的每个键和值
    for key, value_set in dictionary.items():
        # 将值集合添加到列表中
        all_values.extend(value_set)
all_values

In [ ]:
# 删除重复节点
data__ = [node_i for node_i in data if node_i.get('node_id') not in all_values]
data__

In [ ]:
data____ = data__

In [ ]:
# 将关系中的节点合并  删除节点时，建立映射
for i in range(len(data__)):
    if data__[i].get('A_id') in all_values:
        for list_merge_dict in list_merge:
            if data__[i]['A_id'] in next(iter(list_merge_dict.values())):
                data__[i]['A_id'] = str(next(iter(list_merge_dict.keys())))
            else:
                continue
    if data__[i].get('B_id') in all_values:
        for list_merge_dict in list_merge:
            if data__[i]['B_id'] in next(iter(list_merge_dict.values())):
                data__[i]['B_id'] = str(next(iter(list_merge_dict.keys())))  
            else:
                continue

In [ ]:
data__

In [ ]:
# 初始化计数器
counter = 1
# 创建一个新列表来存储更新后的数据
new_data__ = []
id_mapping = {}

for item in data__:
    # 复制当前项，避免直接修改原始数据
    new_item = item.copy()
    # 如果当前项包含node_id，则更新它
    if 'node_id' in new_item:
        # 构造新的 node_id，使用 str.format() 或者 f-string 来格式化编号
        new_id = f"EC_K_{counter:04d}"
        id_mapping[new_item['node_id']] = new_id
        new_item['node_id'] = new_id
        # 更新计数器
        counter += 1
    # 添加更新后的项到新列表
    new_data__.append(new_item)

new_data__

In [ ]:
id_mapping

In [ ]:
for item in new_data__:
    if item.get('A_id') in id_mapping:
        item['A_id'] = id_mapping[item['A_id']]
    if item.get('B_id') in id_mapping:
        item['B_id'] = id_mapping[item['B_id']]

In [ ]:
new_data__

In [ ]:
# # 将更新后的数据写回到文件（可选）
# with open(r'./课程_glm4_merge.json', 'w', encoding='utf-8') as file:
with open(r'./output/课程_glm4_merge.json', 'w', encoding='utf-8') as file:
    json.dump(data__, file, indent=4, ensure_ascii=False)

# 3. 链接 neo4j，生成图谱

输入格式：
```json
    [{
        "name_zh": "隐私权",
        "name_en": "Privacy Rights",
        "explanation": "消费者在电子商务中的个人隐私保护",
        "type": "knowledge",
        "node_id": "EC_K_2872"
    },    {
        "name_zh": "电子签名",
        "name_en": "Electronic Signature",
        "explanation": "用于识别和验证签名人身份的数据电文",
        "type": "knowledge",
        "node_id": "EC_K_2873"
    },    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0002",
        "type": "include"
    },    {
        "A_id": "EC_K_0001",
        "B_id": "EC_K_0003",
        "type": "include"
    }]
```

In [ ]:
# JSON文件路径
json_file_path = r'./output/课程_glm4_merge.json' # 将此路径替换为实际的JSON文件路径

# 连接Neo4j数据库
uri = "bolt://localhost:7687"  # Neo4j数据库URI
username = "neo4j"  # Neo4j数据库用户名
password = "xxx"  # Neo4j数据库密码

driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
# 清空数据库
def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.write_transaction(clear_database)

driver.close()

In [ ]:
# 读取JSON文件内容
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
# 使用Neo4j的Python驱动程序创建会话并写入数据
with driver.session() as session:
    for item in data:
        if item['type'] not in ('include', 'relate', 'order'):
            # 执行创建节点的操作
            try:
                id = item['node_id']
            except KeyError:
                print(item)
            type = item['type']
            try:
                name_zh = item['name_zh']
            except KeyError:
                print(item)
            name_en = item.get('name_en', "")  # 使用get方法避免name_en不存在时引发错误
            try:
                explanation = item['explanation']
            except KeyError:
                print(item)
            # updatetime = item['updatetime']
            # 构建Cypher语句创建节点，并设置属性
            session.run("CREATE (n:Item {id: $id, type: $type, name_zh: $name_zh, name_en: $name_en, explanation: $explanation})",
                            id=id, type=type, name_zh=name_zh, name_en=name_en, explanation=explanation)

        else:
            result_a = session.run("MATCH (n) WHERE n.id = $A_id RETURN n", A_id=item['A_id'])
            result_b = session.run("MATCH (n) WHERE n.id = $B_id RETURN n", B_id=item['B_id'])
            # 检查找到的节点是否存在
            if result_a.single() and result_b.single():
                # 如果两个节点都存在，创建关系
                relationship_type = item['type']
                A_id = item['A_id']
                B_id = item['B_id']
                # 构建Cypher语句，并将关系类型作为字符串的一部分
                cypher_query = (
                    f"MATCH (a), (b) WHERE a.id = $A_id AND b.id = $B_id "
                    f"CREATE (a)-[:{relationship_type}]->(b)"
                )
                session.run(cypher_query, A_id=A_id, B_id=B_id)

            else:
                print(item['A_id'] + item['B_id'] + item['type'] + "One or both nodes do not exist.")
                print(result_a.single())
                print(result_b.single())

# # 关闭Neo4j驱动程序
# driver.close()

## txt2cypher

In [ ]:
from langchain_zhipu import ChatZhipuAI

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

安装 apoc 插件，参考：https://blog.csdn.net/zz_dyx/article/details/135172438

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="xxx"
)

In [ ]:
# os.environ['OPENAI_API_KEY'] = "sk-"
chain = GraphCypherQAChain.from_llm(
    ChatZhipuAI(temperature=0.01), graph=graph, verbose=True,
)

In [ ]:
chain.invoke({'query': """与实体「商业模式」关系为 relate 关系的实体"""})